In [1]:
# %pip install pandas
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm import tqdm

2025-06-14 19:55:16.942167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-14 19:55:17.274993: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-14 19:55:17.275012: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-06-14 19:55:17.315737: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 19:55:18.224357: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv('data.csv', encoding='windows-1252', header=None, usecols=[2, 5])

In [3]:
df['date'] = pd.to_datetime(df[2].astype(str))

/home/wladek/.local/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [4]:
df

,2,5,date
0,Mon Apr 06 22:19:45 PDT 2009,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",2009-04-06 22:19:45
1,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,2009-04-06 22:19:49
2,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,2009-04-06 22:19:53
3,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,2009-04-06 22:19:57
4,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",2009-04-06 22:19:57
...,...,...,...
1599995,Tue Jun 16 08:40:49 PDT 2009,Just woke up. Having no school is the best fee...,2009-06-16 08:40:49
1599996,Tue Jun 16 08:40:49 PDT 2009,TheWDB.com - Very cool to hear old Walt interv...,2009-06-16 08:40:49
1599997,Tue Jun 16 08:40:49 PDT 2009,Are you ready for your MoJo Makeover? Ask me f...,2009-06-16 08:40:49
1599998,Tue Jun 16 08:40:49 PDT 2009,Happy 38th Birthday to my boo of alll time!!! ...,2009-06-16 08:40:49


In [5]:
# Setup pipeline on GPU
fe_pipeline = pipeline(
    task="feature-extraction",
    model="vinai/bertweet-base",
    torch_dtype=torch.float16,
    device=0
)

/home/wladek/.local/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [53]:
from datasets import Dataset
ds = Dataset.from_pandas(df[[5]])
batch_size=32
texts = df[5].tolist()
output_file = "/media/wladek/Wladyslaw/bertweet_embeddings.pt"
with open(output_file, "wb") as f:
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        # Inference
        with torch.no_grad():
            outputs_list = fe_pipeline(batch, padding=True, truncation=True, return_tensors="pt")
            outputs = torch.stack([emb[0, 0, :] for emb in outputs_list])

        emb = torch.tensor(outputs).float().cpu().numpy() # 32 x 768
        f.write(emb.tobytes())  # write raw bytes


  0%|                                                 | 0/50000 [00:00<?, ?it/s]/home/wladek/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  
/tmp/ipykernel_127459/1486891721.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(outputs).float().cpu().numpy() # 32 x 768
100%|███████████████████████████████████| 50000/50000 [7:19:05<00:00,  1.90it/s]


In [6]:
import numpy as np
with open("/media/wladek/Wladyslaw/bertweet_embeddings.pt", "rb") as f:
    # header = np.frombuffer(f.read(8), dtype=np.int32)
    # total_embeddings, embedding_dim = header
    data = f.read()
    embeddings = np.frombuffer(data, dtype=np.float32).reshape(1_600_000, 768)

In [7]:
embeddings.shape

(1600000, 768)

In [20]:
indices = np.random.choice([i for i in range(1_600_000)], size=16_000, replace=False)
embeddings_train = embeddings[indices]

In [22]:
embeddings_train.shape

(16000, 768)

In [25]:
import umap
import numpy as np
import joblib
umap_model = umap.UMAP(
    n_neighbors=5,        # controls local vs global structure
    n_components=3,        # 2D for visualization; use 3 or more if needed
    metric='cosine',       # or 'euclidean'
    n_jobs=-1
).fit(embeddings_train)
result = umap_model.transform(embeddings)
result.dump("umap_result_3d.npy")

In [26]:
result1 = np.load("umap_result_3d.npy", allow_pickle=True)
result1.shape

(1600000, 3)

In [29]:
from pacmap import PaCMAP
pacmap_model = PaCMAP(n_components=2,
                      n_neighbors=10,
                      MN_ratio=0.5,
                      FP_ratio=2.0,
                     save_tree=True)
pacmap_model.fit(embeddings_train)
result = pacmap_model.transform(embeddings)
result.dump("pacmap_result.npy")

In [30]:
result1 = np.load("pacmap_result.npy", allow_pickle=True)
result1.shape

(1600000, 2)

In [31]:
pacmap_model = PaCMAP(n_components=3,
                      n_neighbors=10,
                      MN_ratio=0.5,
                      FP_ratio=2.0,
                     save_tree=True)
pacmap_model.fit(embeddings_train)
result = pacmap_model.transform(embeddings)
result.dump("pacmap_result_3d.npy")

Note: `n_components != 2` have not been thoroughly tested.


In [32]:
result1 = np.load("pacmap_result_3d.npy", allow_pickle=True)
result1.shape

(1600000, 3)

In [36]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=2)
pca_model.fit(embeddings_train)
result = pca_model.transform(embeddings)
result.dump("pca_result.npy")

In [37]:
pca_model = PCA(n_components=3)
pca_model.fit(embeddings_train)
result = pca_model.transform(embeddings)
result.dump("pca_result_3d.npy")

In [41]:
from sklearn.decomposition import IncrementalPCA
pca_model = IncrementalPCA(n_components=2, batch_size=1000)
pca_model.fit(embeddings)
result = pca_model.transform(embeddings)
result.dump("incremental_pca_result.npy")

In [42]:
from sklearn.decomposition import IncrementalPCA
pca_model = IncrementalPCA(n_components=3, batch_size=1000)
pca_model.fit(embeddings)
result = pca_model.transform(embeddings)
result.dump("incremental_pca_result_3d.npy")